In [1]:
#%cd C:/Users/Mathieu/Desktop/Projets/Benter
%cd /home/mathieu/Mindsay/mathieu/Benter-Project

/home/mathieu/Mindsay/mathieu/Benter-Project


In [2]:
%env ENVIRONMENT=development

env: ENVIRONMENT=development


In [3]:
from models.horse import Horse
from models.runner import Runner
from models.race import Race
from database.setup import create_sqlalchemy_session
import tqdm
import random
import re
from tabulate import tabulate
import os
import json
import datetime as dt
from typing import List
import sqlalchemy as sa

from utils.music import parse_music
from utils.logger import setup_logger
from constants import UnibetHorseSex

BATCH_SIZE = 1e5
UNIBET_DATA_PATH = "./data/Unibet"
with create_sqlalchemy_session() as db_session:
    horse=db_session.query(Horse).first()
    
logger = setup_logger(__name__)
    

In [4]:
def is_wrong_age_sequence(ages:List[int])->bool:
    ages = [age for age in ages if age is not None]
    return sorted(ages) != ages

forbidden_sex_transitions = [(UnibetHorseSex.MALE, UnibetHorseSex.FEMALE),
                             (UnibetHorseSex.FEMALE, UnibetHorseSex.MALE), 
                             (UnibetHorseSex.GELDING, UnibetHorseSex.FEMALE), 
                             (UnibetHorseSex.GELDING, UnibetHorseSex.MALE),
                             (UnibetHorseSex.FEMALE, UnibetHorseSex.GELDING)
                            ]

forbidden_sex_transitions_regex = r"|".join([f"{s1.value}{s2.value}" for (s1,s2) in forbidden_sex_transitions])

def is_wrong_sex_sequence(sexes:List[UnibetHorseSex])->bool:
    sexes=[sex.value for sex in sexes if sex !=UnibetHorseSex.UNKNOWN]
    if not sexes:
        return False
    sexes_str ="".join(sexes)
    return bool(re.search(forbidden_sex_transitions_regex, sexes_str))
    

def is_wrong_stake_sequence(stakes:List[int])->bool:
    stakes = [stake for stake in stakes if stake is not None]
    return sorted(stakes) != stakes

# TODO add music check 

In [6]:
start_id=0

wrong_horse_ids_age=set()
wrong_horse_ids_sex=set()
wrong_horse_ids_stake=set()
n_seen_horses=0
with create_sqlalchemy_session() as db_session:
    horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
    for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=BATCH_SIZE):
        if is_wrong_age_sequence(ages=[runner.age for runner in horse.runners]):
            wrong_horse_ids_age.add(horse.id)
            
        if is_wrong_sex_sequence(sexes=[runner.sex for runner in horse.runners]):
            wrong_horse_ids_sex.add(horse.id)
            
        if is_wrong_stake_sequence(stakes=[runner.stakes for runner in horse.runners]):
            wrong_horse_ids_stake.add(horse.id)
            
            
    n_seen_horses+=len(horses)
    while len(horses)==BATCH_SIZE:
        start_id=horses[-1].id
        horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
        for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=len(horses)):
            if is_wrong_age_sequence(ages=[runner.age for runner in horse.runners]):
                wrong_horse_ids_age.add(horse.id)

            if is_wrong_sex_sequence(sexes=[runner.sex for runner in horse.runners]):
                wrong_horse_ids_sex.add(horse.id)

            if is_wrong_stake_sequence(stakes=[runner.stakes for runner in horse.runners]):
                wrong_horse_ids_stake.add(horse.id)
            
        n_seen_horses+=len(horses)

0:  10%|▉         | 9591/100000.0 [00:05<00:51, 1764.43it/s]


In [7]:
n_seen_horses

9591

In [8]:
len(wrong_horse_ids_age)

0

In [9]:
len(wrong_horse_ids_sex)

0

In [10]:
len(wrong_horse_ids_stake)

10

In [14]:
# print runners
filter_recent_dates = False
horse_ids = wrong_horse_ids_stake

with create_sqlalchemy_session() as db_session:
    for horse_id in random.sample(horse_ids, 10):
        runners=db_session.query(Runner).filter(Runner.horse_id==horse_id).all()
        if filter_recent_dates and (runners[0].date<dt.date(2019,1,1)):
            continue
        headers= ["id", "date","race_id","name",   "age", "sex","stakes", "origins"]
        to_print=[]
        for runner in runners:
            to_print.append([runner.id, runner.date,runner.race_id, runner.horse.name, runner.age,runner.sex.name,  runner.stakes, runner.origins])
            
        print(tabulate(to_print, headers=headers))
        
        print()

  id  date          race_id  name       age  sex        stakes  origins
----  ----------  ---------  -------  -----  -------  --------  ----------------
 972  2005-06-24         73  FURIOSO     13  GELDING     33306  Brezzo - Quenice
3142  2005-07-04        249  FURIOSO     13  GELDING       915  Brezzo - Quenice
8030  2005-07-26        620  FURIOSO     13  GELDING     33306  Brezzo - Quenice

  id  date          race_id  name              age  sex       stakes  origins
----  ----------  ---------  --------------  -----  ------  --------  --------------------------
 168  2005-06-20         14  HERE SHE COMES      5  FEMALE     10000  Take Risks - Seule A Paris
2945  2005-07-03        235  HERE SHE COMES      5  FEMALE         0  Take Risks - Seule A Paris

  id  date          race_id  name         age  sex      stakes  origins
----  ----------  ---------  ---------  -----  -----  --------  ------------------
 975  2005-06-24         74  ARAZIETEN      3  MALE      14700  Zieten - Arazi

In [116]:
runner_id =2639713

In [117]:
with create_sqlalchemy_session() as db_session:
    runner = db_session.query(Runner).filter(Runner.id==runner_id).one()
    runner.unibet_code
    runner.race_id


In [118]:
runner.unibet_code

(datetime.date(2020, 2, 22), 5, 7, 3)

In [137]:
date, horse_show_rank, race_rank, runner_rank = dt.date(2020, 11,13), 8, 8, 5 # runner.unibet_code
if not date.isoformat() in os.listdir(UNIBET_DATA_PATH):
    logger.warning("Could not find folder for date: %s", date.isoformat())
    
day_folder_path = os.path.join(UNIBET_DATA_PATH, date.isoformat())
if "programme.json" not in os.listdir(day_folder_path):
    logger.warning(
        "Could not find programme.json for date: %s", date.isoformat()
    )
    

with open(os.path.join(day_folder_path, "programme.json"), "r") as fp:
    programme = json.load(fp=fp)
    
horse_shows_dict = programme["data"]
for horse_show_dict in horse_shows_dict:
    if horse_show_dict['rank']==horse_show_rank:
        for race_dict in horse_show_dict['races']:
            if race_dict['rank']==race_rank:
                for runner_dict in race_dict['runners']:
                    if runner_dict['rank']==runner_rank:
                        print("from programme.json", runner_dict)
print()
                        
race_path = os.path.join(
    day_folder_path,
    f"R{horse_show_rank}_C{race_rank}.json",
)
with open(race_path, "r") as fp:
    complete_race_dict = json.load(fp=fp)
    
for runner_dict in complete_race_dict['runners']:
    if runner_dict['rank']==runner_rank:
        print("from race.json", runner_dict)
        
print()

runner_statistics_path = os.path.join(
day_folder_path,
f"R{horse_show_rank}_"
f"C{race_rank}_"
f"RN{runner_rank}.json",
)
with open(runner_statistics_path, "r") as fp:
    runner_stats = json.load(fp=fp)
print("from runner.json", runner_stats['fiche']['infos_generales'])

from programme.json {'zeturfId': 3304628, 'raceId': 272149, 'rank': 5, 'name': 'SEÃ\x91ORITA ROBERTA (AR)', 'jockey': 'Luciano E. Cabrera', 'weight': 56, 'distance': 0, 'status': '40', 'team': 0, 'draw': 5, 'blinkers': '0', 'shoes': '', 'position': '1', 'silk': '6906_202011130_0805.gif', 'betcounter': 0, 'details': None}

from race.json {'zeturfId': 3304628, 'raceId': 272149, 'rank': 5, 'name': 'SEÃ\x91ORITA ROBERTA (AR)', 'jockey': 'Luciano E. Cabrera', 'weight': 56, 'distance': 0, 'status': '40', 'team': 0, 'draw': 5, 'blinkers': '0', 'shoes': '', 'position': '1', 'silk': '6906_202011130_0805.gif', 'betcounter': 0, 'details': {'zeturfId': 3304628, 'stakes': 0, 'musique': '', 'sex': 'F', 'age': '3', 'owner': 'El Vulcan', 'trainer': 'Marcelo N. Reale', 'coat': 'Gr.', 'origins': 'Flesh For Fantasy-Noche Roberta', 'comment': '', 'time': None, 'reduction': None, 'length': None}}

from runner.json {'corde': 5, 'casaque': '/casaques_ok/G0058852.gif', 'sexe': 'F', 'discipline': 'Plat', 'oeil

In [99]:
complete_race_dict

{'get_url': 'https://www.unibet.fr/zones/turf/race.json?raceId=244977',
 'query_date': '2020-05-03T05:41:03.964704',
 'zeturfId': 244977,
 'meetingId': 39983,
 'starttime': 1578163500000,
 'date': '2020-01-04',
 'type': 'Plat',
 'name': 'Prix Ballistic',
 'rank': 2,
 'conditions': 'Handicap - 3 ans et plus - Sable - Corde à gauche',
 'stake': 5987,
 'arjelLevel': '2',
 'distance': 1900,
 'betCounter': None,
 'friendlyUrl': '/turf/race/04-01-2020-R5-C2-chile-prix-ballistic.html',
 'runners': [{'zeturfId': 2990640,
   'raceId': 244977,
   'rank': 1,
   'name': 'EMPRESARIAL (CL)',
   'jockey': 'Medina Ja.',
   'weight': 52,
   'distance': 0,
   'status': '40',
   'team': 0,
   'draw': 1,
   'blinkers': '0',
   'shoes': '',
   'position': '1',
   'silk': '6967_202001040_0201.gif',
   'betcounter': 0,
   'details': {'zeturfId': 2990640,
    'stakes': 0,
    'musique': '(19) 6p 1p 5p 2p 6p 0p 1p 0p 2p 7p 6p 0p ',
    'sex': 'H',
    'age': '7',
    'owner': 'Maquehua',
    'trainer': 'Ivan G

In [85]:
runner_stats

{'get_url': 'https://www.unibet.fr/zones/turf/statistiques.json?raceId=248180&runnerRank=3',
 'query_date': '2020-11-23T05:08:37.653888',
 'fiche': {'infos_generales': {'corde': 3,
   'casaque': '/casaques_ok/G0186249.gif',
   'sexe': 'M',
   'discipline': 'Plat',
   'oeillere': '',
   'jockey': 'Nelson Enrique FIGUEROA MARTINEZ',
   'nom': 'CIZAnERO {CHL}',
   'gain': 0,
   'proprietaire': 'La Tropa',
   'entraineur': 'TAPIA S. S.',
   'robe': '',
   'musique': '4p 5p 0p 1p 8p 4p 2p 4p 1p 5p 0p 6p',
   'record': '',
   'ferrure': 0,
   'age': 2016,
   'parents': 'Wildcat Spirit - La Kadima'},
  'perfs': {'course': [{'date': '2020-10-22',
     'nom_course': 'PRIX Floreado',
     'prix': 3698,
     'partant': {'cheval': [{'corde': 5,
        'num_cheval': 5,
        'distance': 0,
        'handicap': '',
        'poids': 56,
        'classement': 1,
        'oeillere': '',
        'tx_recla': '',
        'nom': 'QUASSAR {CHL}',
        'jockey': 'Diego N. Sanchez',
        'temps': '',


In [62]:
runner_stats.keys()

dict_keys(['get_url', 'query_date', 'fiche'])

In [87]:
runner.race_id

209069

In [88]:
# Describe a race
race_id = 209069

with create_sqlalchemy_session() as db_session:
    race = db_session.query(Race).filter(Race.id==race_id).one()
    print(tabulate([[race.id, race.date.isoformat(), race.unibet_code, race.type.value]],headers=["id", "date", "unibet_code", "type"], tablefmt="grid"))
    print("conditions", race.conditions)
    print()
    headers= ["id", "date","race_id","name", "age","sex", "stakes", "hist_stakes", "origins"]
    to_print=[]
    for runner in race.runners:
        to_print.append([runner.id, runner.date,runner.race_id, runner.horse.name, runner.age,runner.sex.name,  runner.stakes, runner.historical_stakes, runner.origins])

    print(tabulate(to_print, headers=headers, tablefmt="grid"))


+--------+------------+------------------------------------+--------+
|     id | date       | unibet_code                        | type   |
+========+============+====================================+========+
| 209069 | 2020-02-22 | (datetime.date(2020, 2, 22), 5, 7) | Plat   |
+--------+------------+------------------------------------+--------+
conditions 3 ans et au-dessus - Dirt - Corde à Gauche

+---------+------------+-----------+-------------------+-------+---------+----------+---------------+-------------------------------+
|      id | date       |   race_id | name              |   age | sex     |   stakes | hist_stakes   | origins                       |
+=========+============+===========+===================+=======+=========+==========+===============+===============================+
| 2639711 | 2020-02-22 |    209069 | TAVUCHO           |    10 | GELDING |        0 |               | Pel - De La Mancha            |
+---------+------------+-----------+-------------------+---

'Plat'

In [6]:
start_id=0
with create_sqlalchemy_session() as db_session:

    horses=db_session.query(Horse).filter(Horse.id>start_id).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
    
    for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=BATCH_SIZE):
        origins={runner.origins for runner in horse.runners if runner.origins is not None}
        if not origins:
            continue
        if len(origins)==2:
            continue
        print(origins, horse.id)
        break

0:   0%|          | 0/100000.0 [00:00<?, ?it/s]


In [7]:
a="Turgeon-Shannon"

In [8]:
father_name, mother_name=re.split("[-/]",a)

In [9]:
father_name=father_name.upper()
mother_name=mother_name.upper()

In [10]:
with create_sqlalchemy_session() as db_session:

    father_horse_candidates=db_session.query(Horse).filter(Horse.name==father_name).all()
    
    mother_horse_candidates=db_session.query(Horse).filter(Horse.name==mother_name).all()

In [11]:
mother_horse_candidates

[]

In [4]:
start_id=0
# TODO female names, male_names
not_understood_parent_names = set()
horse_names=set()
n_seen_horses=0
with create_sqlalchemy_session() as db_session:
    horses=db_session.query(Horse).filter(Horse.id>start_id, Horse.father_id.is_(None), Horse.mother_id.is_(None)).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
    for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=BATCH_SIZE):
        horse_names.add(horse.name)
        for runner in horse.runners:
            not_understood_parent_names.add(runner.origins)
    
            
    n_seen_horses+=len(horses)
    while len(horses)==BATCH_SIZE:
        start_id=horses[-1].id
        horses=db_session.query(Horse).filter(Horse.id>start_id, Horse.father_id.is_(None), Horse.mother_id.is_(None)).order_by(Horse.id.asc()).limit(BATCH_SIZE).all()
        for horse in tqdm.tqdm(horses, desc=f'{start_id}', total=len(horses)):
            horse_names.add(horse.name)
            for runner in horse.runners:
                not_understood_parent_names.add(runner.origins)
        n_seen_horses+=len(horses)

0:  55%|█████▌    | 55176/100000.0 [00:36<00:29, 1532.60it/s]


In [5]:
not_understood_parent_names.discard('')
not_understood_parent_names.discard("-")

In [6]:
not_understood_parent_names

{'A P Indy Brandy Rose',
 "Acambaro Siberian's Image",
 'Admire Vega Greetings',
 'Aelan Hapi Romy De Baruthet',
 'Alesso Tahis De Farges',
 'Alesso Valinka II',
 'Alhaarth Partie De Dames',
 'Ali-Royal (IRE) / Grandeur And Grace (USA)',
 'Ali-Royal/Byliny',
 'Almutawakel Terracotta Hut',
 'Alphabet Soup Melina Mou',
 'Alzao-Ela-aristokratissa',
 'Anabaa Amitie Fatale',
 'Anabaa Mille Miglia',
 'Anabaa Miss Mat',
 'Anabaa Pyramid Lake',
 'Anabaa Spectacular Joke',
 'Anabaa Value For Money',
 'Anees Russian Flight',
 'Anshan-Glory-glory',
 'Anshan/Glory-Glory',
 'Apeldoorn Tots Off',
 "Apple Tree L'Apparition",
 'Apple Tree Lady Burlington',
 'Apple Tree Pom',
 'Apple Tree Soie Moriniere',
 'April Night Stadia',
 "Arnaqueur Corail D'Haguenet",
 'Arnaqueur Diana De Beaufai',
 'Ashkalani Finir En Beaute',
 'Assessor Enigma II',
 'Astair Black Charm',
 'Atticus Ring Beaune',
 'Attune Blanche Dame',
 'Attune Noble Folly',
 'Attune Slew Dancer',
 'B.w.t.magic* (FI)-Rose-mary Aspen (FI)',
 'B

In [25]:
# Trying to find existing horse_name in parent_names

parent_name_mapping ={}
for parent_name in tqdm.tqdm(not_understood_parent_names, total=len(not_understood_parent_names)):
    matching_mother_names =[]
    matching_father_names=[]
    for horse_name in horse_names:
        if "(" in horse_name:
            continue
        if re.search(fr"\b{horse_name.upper()}$", parent_name.upper()):
            matching_mother_names.append(horse_name)
        if re.search(fr"^{horse_name.upper()}\b", parent_name.upper()):
            matching_father_names.append(horse_name)
    matched_mother_name = None
    if matching_mother_names:
        matched_mother_name=max(matching_mother_names, key=len)
    matched_father_name =None
    if matching_father_names:
        matching_father_names= max(matching_father_names, key=len)
        
    if (len(matched_mother_name or "") + len(matching_father_names or ""))>=len(parent_name):
        print(f"Error with {parent_names}")
        continue
    if matched_mother_name and matched_father_name:
        
        continue
        
    if matched_mother_name and matched_father_name is None:
        


  0%|          | 1/575 [00:07<1:10:59,  7.42s/it]

Robin Des Pres Nik Bey NIK BEY
Robin Des Pres Nik Bey ROBIN DES PRES


  0%|          | 2/575 [00:14<1:08:56,  7.22s/it]

Rainbow Quest Sarliya SARLIYA


  1%|          | 4/575 [00:27<1:06:45,  7.01s/it]

...

Murmure Blade Des Garennes BLADE DES GARENNES
Murmure Blade Des Garennes MURMURE


100%|█████████▉| 574/575 [1:12:14<00:09,  9.76s/it]

Perrault Clyde Des Mottes CLYDE DES MOTTES
Perrault Clyde Des Mottes PERRAULT


100%|██████████| 575/575 [1:12:23<00:00,  7.55s/it]

Kingmambo Legend Maker LEGEND MAKER


In [22]:
with create_sqlalchemy_session() as db_session:
    horse=db_session.query(Horse).filter(Horse.name=="SARLIYA").one()

In [24]:
horse.is_born_male

False

In [ ]:
# Looking at ngram

In [28]:
from collections import Counter

ModuleNotFoundError: No module named 'nltk'

In [29]:
# https://stackoverflow.com/questions/42373747/is-there-a-more-efficient-way-to-find-most-common-n-grams
def ngrams(text, n=2):
     return zip(*[text[i:] for i in range(n)])

In [48]:
for N_GRAM in range(10,3,-1):
    ngram_counts=Counter()
    for parent_name in tqdm.tqdm(not_understood_parent_names,total=len(not_understood_parent_names)) :
        ngram_counts += Counter(ngrams(parent_name.split(), N_GRAM))
    print(N_GRAM, max(ngram_counts.values()) if ngram_counts else 0)

    

100%|██████████| 526/526 [00:00<00:00, 41309.26it/s]

10 0
9 0
8 0
7 1
6 1
5 1
4 2


In [49]:
ngram_counts.most_common(10)

[(('Marchand', 'De', 'Sable', 'Lady'), 2),
 (('Lute', 'Antique', 'Gold', 'Wind'), 1),
 (('Green', 'Desert', 'Princesse', 'Bilbao'), 1),
 (('De', 'Sable', 'Lady', 'Petronille'), 1),
 (('Solid', 'Illusion', 'Dark', 'Kay'), 1),
 (('Jeune', 'Homme', 'Little', 'Deep'), 1),
 (('Homme', 'Little', 'Deep', 'Water'), 1),
 (('Lord', 'Of', 'Men', 'Nimes'), 1),
 (('Linamix', 'Broad', 'And', 'High'), 1),
 (('Cricket', 'Ball', 'Maria', 'Star'), 1)]

In [12]:
from collections import namedtuple
from typing import List

MusicEvent = namedtuple("MusicEvent", ["rank", "race_type", "year"])


def parse_unibet_music(current_year:int, ) ->List[MusicEvent]:

    music_events:List[MusicEvent]=[]
    events=re.findall(r"(([ATRDN]|\d)([a-z]))|(\((\d{1,2})\))", runner.music)
    for event in events:
        race_event, rank, race_type, _, year = event
        if year:
            assert not race_event
            next_current_year = int(f'20{year}')
            # We read music from left to right to go backward in time
            assert next_current_year < current_year
            current_year=next_current_year
            continue
        assert not year
        music_events.append(MusicEvent(rank=rank, race_type=race_type, year=current_year))
        
    return music_events
    

In [13]:
music_events

NameError: name 'music_events' is not defined

In [ ]:
runner.music

In [ ]:
race_type

In [ ]:
_

In [ ]:
year

In [ ]:
race_type

In [1]:
# Appendix

In [2]:
import logging
from collections import Counter
import datetime as dt
from typing import Optional, Generator
from constants import UNIBET_MIN_DATE
from tqdm import tqdm
import os 
import json

logger = logging.getLogger(__name__)
UNIBET_DATA_PATH = "./data/Unibet"

coats=Counter()
coats_info = Counter()
co_coats=Counter()
def date_countdown_generator(
    start_date: dt.date, end_date: Optional[dt.date]
) -> Generator[dt.date, None, None]:
    end_date = end_date or dt.date.today()
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += dt.timedelta(days=1)
        

for date in tqdm(
    date_countdown_generator(
        start_date=UNIBET_MIN_DATE,
        end_date=dt.date.today() - dt.timedelta(days=1),
    ),
    total=(dt.date.today() - dt.timedelta(days=1) - UNIBET_MIN_DATE).days,
    unit="days",
):
    if not date.isoformat() in os.listdir(UNIBET_DATA_PATH):
        logger.info("Could not find folder for date: %s", date.isoformat())
        continue
    day_folder_path = os.path.join(UNIBET_DATA_PATH, date.isoformat())
    if "programme.json" not in os.listdir(day_folder_path):
        logger.info(
            "Could not find programme.json for date: %s", date.isoformat()
        )
        continue

    with open(os.path.join(day_folder_path, "programme.json"), "r") as fp:
        programme = json.load(fp=fp)
    if "data" not in programme:
        logger.info("Can not import programme of %s", date.isoformat())
        continue

    horse_shows_dict = programme["data"]
    for horse_show_dict in horse_shows_dict:

        if horse_show_dict.get("races"):

            for race_dict in horse_show_dict["races"]:
                race_path = os.path.join(
                    day_folder_path,
                    f"R{horse_show_dict['rank']}_C" f"{race_dict['rank']}.json",
                )
                with open(race_path, "r") as fp:
                    complete_race_dict = json.load(fp=fp)

                current_race_dict = complete_race_dict
                if complete_race_dict.get("note") == "server error, no json":
                    # Can not use complete_race
                    current_race_dict = race_dict

                for runner_dict in current_race_dict["runners"]:
                    coat=runner_dict["details"].get("coat") if runner_dict.get("details") else None
                    coats[coat]+=1
                    
                    runner_statistics_path = os.path.join(
                                day_folder_path,
                                f"R{horse_show_dict['rank']}_"
                                f"C{race_dict['rank']}_"
                                f"RN{runner_dict['rank']}.json",
                            )
                    with open(runner_statistics_path, "r") as fp:
                        runner_stats = json.load(fp=fp)
                        
                    runner_stats_info: Optional[dict] = runner_stats.get("fiche", {}).get("infos_generales")
                    coat_info=runner_stats_info.get("robe") if runner_stats_info else None
                    coats_info[coat_info]+=1
                    
                    co_coats[(coat, coat_info)]+=1

5660days [16:58,  5.56days/s]                        


In [3]:
coats

Counter({'GR': 6809,
         'B. Fonce': 189960,
         'AL': 12396,
         'B': 33435,
         'BF': 6965,
         'B.': 1385065,
         'Al.': 475192,
         'Noi.P': 3342,
         'Noir': 114033,
         'Al. B': 823,
         'Bb.': 4774,
         'B. Fo': 5158,
         '': 379327,
         'B. Clair': 5836,
         'Al. Fonce': 6189,
         'Noi.Pan': 683,
         'BB': 4370,
         'Gr.': 91924,
         'RO': 292,
         'AB': 986,
         'AC': 48,
         'BC': 128,
         'GF': 291,
         'NO': 438,
         'B. Ce': 22,
         'Al. F': 438,
         None: 3579,
         'Bb. Fonce': 326,
         'Bb. F': 216,
         'Al. Brule': 244,
         'Al. Cuivre': 23,
         'Al. C': 151,
         'Bay': 414,
         'AF': 452,
         'Rouan': 2530,
         'B. Rubican': 36580,
         'Al. Rub.': 47,
         'Brown': 15152,
         'B. Fonc': 20,
         'BM': 79,
         'Al. Cr.Lav': 25,
         'Al': 10,
         'AM': 50,
         '

In [4]:
coats_info

Counter({'GR': 21926,
         'BF': 20836,
         'AL': 34468,
         'B': 109000,
         None: 207618,
         'Al.': 26889,
         'B.': 464628,
         'B. Fo': 13078,
         'B. FONCE': 53073,
         'Noi.P': 8486,
         'Noir': 3410,
         'B. FO': 7343,
         'Al. B': 1984,
         'Bb.': 1829,
         'Bb. F': 996,
         'B. Cl': 582,
         'B. Ru': 55,
         'AL.': 264221,
         'NOI.P': 5957,
         'Al. F': 793,
         'NOI.PAN': 70887,
         'BB': 13196,
         'RO': 1098,
         'AB': 2725,
         'Bai': 54056,
         'AC': 96,
         'NO': 1163,
         'B. Ce': 44,
         'NOIR': 17009,
         'Al. C': 416,
         'B. CLAIR': 1767,
         'BC': 512,
         'AF': 777,
         'GF': 976,
         'Gr': 5251,
         'Al': 18952,
         'B. CR': 725,
         'Al. R': 166,
         '': 905897,
         'Gr.': 202,
         'Rouan': 68,
         'b': 24293,
         'AM': 162,
         'AL. F': 648,
       

In [5]:
co_coats

Counter({('GR', 'GR'): 5958,
         ('B. Fonce', 'BF'): 14024,
         ('AL', 'AL'): 10809,
         ('B', 'B'): 29005,
         ('BF', 'BF'): 6208,
         ('B.', 'B'): 76700,
         ('B', None): 4171,
         ('Al.', 'Al.'): 26601,
         ('B.', 'B.'): 461073,
         ('Al.', 'B.'): 106,
         ('AL', 'B. Fo'): 24,
         ('B. Fonce', 'B. FONCE'): 52374,
         ('Noi.P', 'Noi.P'): 3203,
         ('Noir', 'Noir'): 3275,
         ('B. Fonce', 'B. FO'): 7204,
         ('Al. B', 'Al. B'): 772,
         ('Bb.', 'Bb.'): 1410,
         ('B. Fo', 'B. Fo'): 4873,
         ('Bb.', 'Bb. F'): 46,
         ('B.', None): 92134,
         ('', 'B.'): 2213,
         ('B. Fonce', 'B. Fo'): 7766,
         ('B. Clair', 'B. Cl'): 64,
         ('', 'B. Ru'): 18,
         ('Al.', 'AL.'): 263288,
         ('Noir', 'NOI.P'): 5909,
         ('Noir', 'Noi.P'): 4566,
         ('Al. Fonce', 'Al. F'): 54,
         ('Noi.Pan', 'Noi.P'): 650,
         ('Noir', 'NOI.PAN'): 70597,
         ('BB', 'BB'

In [13]:
list(set(coats.keys()).union(set(coats_info.keys())))

['',
 'Bai C',
 'AC',
 'BAI B',
 'BG',
 'grey-bay',
 'B. Clair',
 'BF',
 'NOIR',
 'B. CR',
 'A.',
 'ROUAN',
 'b/br',
 'AL.MELANGE',
 'B.Foncé',
 'CHOCOLAT',
 'BR',
 'AL.MEL.',
 'bb.',
 'B',
 'Noi.P',
 'AL. BRULE',
 'NO',
 'AR',
 'AL',
 'BAI',
 'AA',
 'AL. B',
 'AF',
 'BAI FONCE',
 'Noi.',
 'GR.',
 'BM',
 'GC',
 'gr',
 'B. Fonce',
 'Al. R',
 'Bay',
 'B.d',
 'Bai P',
 'Palomino',
 'R.',
 'Al. C',
 'B. Cerise',
 'AL. F',
 'dkb/br',
 'grey-blk',
 'BB. F',
 'AU',
 'GREYROAN',
 'CHOCO',
 'Chestnut',
 'Al. B',
 'Noi.Pan',
 ' Al.',
 'br',
 'Brown',
 'AB',
 'AL. CU',
 'B. CL',
 'Louve',
 'SM',
 'bai',
 'AL. P',
 'marron',
 's',
 'GRIS',
 None,
 'G.',
 'N.',
 'Bb. Fonce',
 'Chocolat',
 'Bb.',
 'B. Fo',
 'AM',
 'PALOMINO',
 'Gris',
 'Al. Clair',
 'Bl',
 'Choco',
 'grey-chest',
 'Bai',
 'grey-brown',
 'B.',
 'ro',
 'B. Foncé',
 'BAI CLAIR',
 'B. Ru',
 'Al. Aub.',
 'Al. Rub.',
 'AUB.',
 'b/gr',
 ' B.',
 'Gray',
 'BAYBROWN',
 'brown',
 'GR. F',
 'MO',
 'NOI.P',
 'AL. C',
 'Bf.',
 'RG',
 'BBF',
 'ALZ

In [14]:
import logging
from collections import Counter
import datetime as dt
from typing import Optional, Generator
from constants import UNIBET_MIN_DATE
from tqdm import tqdm
import os 
import json

logger = logging.getLogger(__name__)
UNIBET_DATA_PATH = "./data/Unibet"

stakes_equal = Counter()
different_stakes=[]
def date_countdown_generator(
    start_date: dt.date, end_date: Optional[dt.date]
) -> Generator[dt.date, None, None]:
    end_date = end_date or dt.date.today()
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += dt.timedelta(days=1)
        

for date in tqdm(
    date_countdown_generator(
        start_date=UNIBET_MIN_DATE,
        end_date=dt.date.today() - dt.timedelta(days=1),
    ),
    total=(dt.date.today() - dt.timedelta(days=1) - UNIBET_MIN_DATE).days,
    unit="days",
):
    if not date.isoformat() in os.listdir(UNIBET_DATA_PATH):
        logger.info("Could not find folder for date: %s", date.isoformat())
        continue
    day_folder_path = os.path.join(UNIBET_DATA_PATH, date.isoformat())
    if "programme.json" not in os.listdir(day_folder_path):
        logger.info(
            "Could not find programme.json for date: %s", date.isoformat()
        )
        continue

    with open(os.path.join(day_folder_path, "programme.json"), "r") as fp:
        programme = json.load(fp=fp)
    if "data" not in programme:
        logger.info("Can not import programme of %s", date.isoformat())
        continue

    horse_shows_dict = programme["data"]
    for horse_show_dict in horse_shows_dict:

        if horse_show_dict.get("races"):

            for race_dict in horse_show_dict["races"]:
                race_path = os.path.join(
                    day_folder_path,
                    f"R{horse_show_dict['rank']}_C" f"{race_dict['rank']}.json",
                )
                with open(race_path, "r") as fp:
                    complete_race_dict = json.load(fp=fp)

                current_race_dict = complete_race_dict
                if complete_race_dict.get("note") == "server error, no json":
                    # Can not use complete_race
                    current_race_dict = race_dict

                for runner_dict in current_race_dict["runners"]:
                    stake=runner_dict["details"].get("stakes") if runner_dict.get("details") else None

                    
                    runner_statistics_path = os.path.join(
                                day_folder_path,
                                f"R{horse_show_dict['rank']}_"
                                f"C{race_dict['rank']}_"
                                f"RN{runner_dict['rank']}.json",
                            )
                    with open(runner_statistics_path, "r") as fp:
                        runner_stats = json.load(fp=fp)
                        
                    runner_stats_info: Optional[dict] = runner_stats.get("fiche", {}).get("infos_generales")
                    stake_info=runner_stats_info.get("gain") if runner_stats_info else None
                    stakes_equal[stake==stake_info]+=1
                    if stake!= stake_info:
                        different_stakes.append((stake, stake_info,date.isoformat(), horse_show_dict['rank'], race_dict['rank'], runner_dict['rank']))

5660days [17:19,  5.45days/s]                        


In [15]:
stakes_equal

Counter({False: 2286852, True: 503495})

In [ ]:
different_stakes

In [125]:
import logging
from collections import Counter
import datetime as dt
from typing import Optional, Generator
from constants import UNIBET_MIN_DATE
from tqdm import tqdm
import os 
import json
from scripts.generate_unibet import _extract_name_country

logger = logging.getLogger(__name__)
UNIBET_DATA_PATH = "./data/Unibet"

equal_names = Counter()
different_names=[]
def date_countdown_generator(
    start_date: dt.date, end_date: Optional[dt.date]
) -> Generator[dt.date, None, None]:
    end_date = end_date or dt.date.today()
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += dt.timedelta(days=1)
        

for date in tqdm(
    date_countdown_generator(
        start_date=UNIBET_MIN_DATE,
        end_date=dt.date.today() - dt.timedelta(days=1),
    ),
    total=(dt.date.today() - dt.timedelta(days=1) - UNIBET_MIN_DATE).days,
    unit="days",
):
    if not date.isoformat() in os.listdir(UNIBET_DATA_PATH):
        logger.info("Could not find folder for date: %s", date.isoformat())
        continue
    day_folder_path = os.path.join(UNIBET_DATA_PATH, date.isoformat())
    if "programme.json" not in os.listdir(day_folder_path):
        logger.info(
            "Could not find programme.json for date: %s", date.isoformat()
        )
        continue

    with open(os.path.join(day_folder_path, "programme.json"), "r") as fp:
        programme = json.load(fp=fp)
    if "data" not in programme:
        logger.info("Can not import programme of %s", date.isoformat())
        continue

    horse_shows_dict = programme["data"]
    for horse_show_dict in horse_shows_dict:

        if horse_show_dict.get("races"):

            for race_dict in horse_show_dict["races"]:
                race_path = os.path.join(
                    day_folder_path,
                    f"R{horse_show_dict['rank']}_C" f"{race_dict['rank']}.json",
                )
                with open(race_path, "r") as fp:
                    complete_race_dict = json.load(fp=fp)

                current_race_dict = complete_race_dict
                if complete_race_dict.get("note") == "server error, no json":
                    # Can not use complete_race
                    current_race_dict = race_dict

                for runner_dict in current_race_dict["runners"]:
                    name=_extract_name_country(runner_dict["name"])[0]

                    
                    runner_statistics_path = os.path.join(
                                day_folder_path,
                                f"R{horse_show_dict['rank']}_"
                                f"C{race_dict['rank']}_"
                                f"RN{runner_dict['rank']}.json",
                            )
                    with open(runner_statistics_path, "r") as fp:
                        runner_stats = json.load(fp=fp)
                        
                    runner_stats_info: Optional[dict] = runner_stats.get("fiche", {}).get("infos_generales")
                    name_info=runner_stats_info.get("nom") if runner_stats_info else None
                    name_info=_extract_name_country(name_info)[0]
                    equal_names[name==name_info]+=1
                    if name!=name_info:
                        different_names.append((name, name_info,date.isoformat(), horse_show_dict['rank'], race_dict['rank'], runner_dict['rank']))

5665days [17:26,  5.41days/s]                        


In [126]:
equal_names

Counter({True: 2564253, False: 226094})

In [128]:
different_names[-5:]

[('IKURO JIEL', None, '2020-11-21', 8, 1, 8),
 ('ECLAIR LUDOIS', None, '2020-11-21', 8, 2, 10),
 ('DIANE DU NORD', None, '2020-11-21', 8, 2, 14),
 ('EPONA DU PASSAGE', None, '2020-11-21', 8, 6, 15),
 ('DESTINO DODVILLE', None, '2020-11-21', 8, 7, 10)]

In [131]:
different_names_not_null_info_name = [dfn for dfn in different_names if dfn[1] is not None]

In [133]:
len(different_names_not_null_info_name)

18426

In [135]:
different_names_not_null_info_name[-20:]

[('SUPER REGALONA', 'WONDER GOLLY', '2020-10-30', 7, 3, 10),
 ('ADRIANKA', 'BABY GOLD', '2020-10-30', 7, 3, 11),
 ('COMO UNA NINA', 'CAPACITA', '2020-10-30', 7, 3, 12),
 ('ESCOLTADORA', 'LARA CROFT', '2020-10-30', 7, 4, 1),
 ('DALE FLORA', 'CONTENIDA', '2020-10-30', 7, 4, 3),
 ('GALICADA STREAPS', 'MANDALA', '2020-10-30', 7, 4, 4),
 ('GRACIAS POR VENIR', 'MILKYWAY', '2020-10-30', 7, 4, 5),
 ('LA FLOR DE ORO', 'QUE COQUETA', '2020-10-30', 7, 4, 6),
 ('EASY CHARITY', 'DICHELLE', '2020-10-30', 7, 4, 7),
 ('WINDSTOÃF', 'WINDSTOÆ', '2020-11-01', 2, 5, 7),
 ('LIEBLING E  D4', 'LIEBLING', '2020-11-04', 5, 8, 5),
 ('GLAÇADA', 'GLAǁDA', '2020-11-08', 6, 5, 8),
 ('SEÃ\x91ORITA ROBERTA', 'SENORITA ROBERTA', '2020-11-13', 8, 8, 5),
 ('MALIKADO', 'EL GRAN CARATA SANT', '2020-11-13', 8, 11, 8),
 ('HÜBSCH TF', 'H܂SCH TF', '2020-11-14', 5, 7, 2),
 ('GLAÇADA', 'GLAǁDA', '2020-11-15', 2, 6, 14),
 ('NADA MÃIS', 'NADA MÉS', '2020-11-15', 3, 6, 2),
 ('JR WALKER E', 'JR WALKER', '2020-11-18', 5, 6, 4),
 ('L

In [138]:
#  ('LA FLOR DE ORO', 'QUE COQUETA', '2020-10-30', 7, 4, 6),

from Levenshtein import distance


In [139]:
distance('LA FLOR DE ORO','QUE COQUETA')

12

In [140]:
distance('HÜBSCH TF', 'H܂SCH TF')

2

In [141]:
distance('JR WALKER E', 'JR WALKER')

2

In [143]:
distance('WINDSTOÃF', 'WINDSTOÆ')

2